In [3]:
import pandas as pd

#from sentence_transformers import SentenceTransformer
import numpy as np
#model = SentenceTransformer('bert-base-nli-mean-tokens')


In [4]:
true = pd.read_csv('C:/Users/katya/Fake_News/News_dataset/True.csv') 
fake = pd.read_csv('C:/Users/katya/Fake_News/News_dataset/Fake.csv')

In [5]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [21]:
sentences = true['title'].unique()
sentences

array(['As U.S. budget fight looms, Republicans flip their fiscal script',
       'U.S. military to accept transgender recruits on Monday: Pentagon',
       "Senior U.S. Republican senator: 'Let Mr. Mueller do his job'",
       ..., 'Minsk cultural hub becomes haven from authorities',
       'Vatican upbeat on possibility of Pope Francis visiting Russia',
       'Indonesia to buy $1.14 billion worth of Russian jets'],
      dtype=object)

In [5]:
sentence_embeddings = model.encode(sentences)
print(sentence_embeddings)

[[-0.5992365   0.45873383 -0.34349346 ... -0.33107406  0.3193629
   0.02686699]
 [ 0.06963822  0.53655505  0.45924962 ...  0.5513485  -0.01077611
   0.25133017]
 [-0.8980431   0.84262395 -0.13861199 ...  0.46117136  0.36786246
   0.04237468]
 ...
 [-0.0926037   0.30926335  1.3640616  ... -0.36090213 -1.3226742
  -0.08313003]
 [-0.48847264  0.21804309  1.414586   ... -0.13482669 -0.00437224
   0.6647935 ]
 [-0.26317427  0.5436258  -0.9435161  ... -0.31720433  0.6195451
   0.79723614]]


In [6]:
sentence_df = pd.DataFrame()
sentence_arr = []
embedding_arr = []

i = 0

for sentence, embedding in zip(sentences, sentence_embeddings):
    #print("Sentence:", sentence)
    #print("Embedding:", embedding)
    #print("")    
    sentence_arr.append(sentences[i])
    embedding_arr.append(embedding)
    i = i + 1

sentence_df['text'] = sentence_arr
sentence_df['emb'] = embedding_arr
#sentence_df['sources'] = sources
print(sentence_df)

                                                    text  \
0      As U.S. budget fight looms, Republicans flip t...   
1      U.S. military to accept transgender recruits o...   
2      Senior U.S. Republican senator: 'Let Mr. Muell...   
3      FBI Russia probe helped by Australian diplomat...   
4      Trump wants Postal Service to charge 'much mor...   
...                                                  ...   
20821  North Korea shipments to Syria chemical arms a...   
20822  LexisNexis withdrew two products from Chinese ...   
20823  Minsk cultural hub becomes haven from authorities   
20824  Vatican upbeat on possibility of Pope Francis ...   
20825  Indonesia to buy $1.14 billion worth of Russia...   

                                                     emb  
0      [-0.5992365, 0.45873383, -0.34349346, 0.103394...  
1      [0.069638215, 0.53655505, 0.45924962, 0.224080...  
2      [-0.8980431, 0.84262395, -0.13861199, 0.363072...  
3      [-0.0060276706, 0.3452367, -0.076539

In [7]:
truedata = sentence_df

In [8]:
csvpath = 'C:/Users/katya/Fake_News/true_emb.csv'
truedata.to_csv(csvpath)    

In [9]:
from nltk.cluster import KMeansClusterer
import nltk

def clustering_question(data,NUM_CLUSTERS = 2):

    sentences = data['text']

    X = np.array(data['emb'].tolist())

    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=25,avoid_empty_clusters=True)

    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

    data['cluster'] = pd.Series(assigned_clusters, index=data.index)
    data['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])

    return data, assigned_clusters

def distance_from_centroid(row):
    # type of emb and centroid is different, hence using tolist below
    return distance_matrix([row['emb']], [row['centroid'].tolist()])[0][0]

# Compute centroid distance to the data
#sentence_df['distance_from_centroid'] = sentence_df.apply(distance_from_centroid, axis=1)

clustering_question(sentence_df)

(                                                    text  \
 0      As U.S. budget fight looms, Republicans flip t...   
 1      U.S. military to accept transgender recruits o...   
 2      Senior U.S. Republican senator: 'Let Mr. Muell...   
 3      FBI Russia probe helped by Australian diplomat...   
 4      Trump wants Postal Service to charge 'much mor...   
 ...                                                  ...   
 20821  North Korea shipments to Syria chemical arms a...   
 20822  LexisNexis withdrew two products from Chinese ...   
 20823  Minsk cultural hub becomes haven from authorities   
 20824  Vatican upbeat on possibility of Pope Francis ...   
 20825  Indonesia to buy $1.14 billion worth of Russia...   
 
                                                      emb  cluster  \
 0      [-0.5992365, 0.45873383, -0.34349346, 0.103394...        0   
 1      [0.069638215, 0.53655505, 0.45924962, 0.224080...        1   
 2      [-0.8980431, 0.84262395, -0.13861199, 0.363072..

In [16]:
filter = sentence_df.cluster.eq(0)
#print(sentence_df[filter]['text'])

array = sentence_df[filter]['text']
print(array.values)
print(len(array))

['As U.S. budget fight looms, Republicans flip their fiscal script'
 'Man says he delivered manure to Mnuchin to protest new U.S. tax law'
 'Virginia officials postpone lottery drawing to decide tied statehouse election'
 ...
 'North Korea shipments to Syria chemical arms agency intercepted: U.N. report'
 'LexisNexis withdrew two products from Chinese market'
 'Minsk cultural hub becomes haven from authorities']
10053


In [11]:
truedata_cluster = sentence_df

In [12]:
truedata_cluster.to_csv('C:/Users/katya/Fake_News/true_cluster.csv')    

In [22]:
sentence_df

,text,emb,cluster,centroid,subject,date
0,WASHINGTON (Reuters) - The head of a conservat...,"[-0.5992365, 0.45873383, -0.34349346, 0.103394...",0,"[-0.10262637, 0.5293654, 0.35757658, 0.0901561...",politicsNews,"December 31, 2017"
1,WASHINGTON (Reuters) - Transgender people will...,"[0.069638215, 0.53655505, 0.45924962, 0.224080...",1,"[-0.24427138, 0.36195734, 0.5969044, 0.1546584...",politicsNews,"December 29, 2017"
2,WASHINGTON (Reuters) - The special counsel inv...,"[-0.8980431, 0.84262395, -0.13861199, 0.363072...",1,"[-0.24427138, 0.36195734, 0.5969044, 0.1546584...",politicsNews,"December 31, 2017"
3,WASHINGTON (Reuters) - Trump campaign adviser ...,"[-0.0060276706, 0.3452367, -0.07653967, 0.5917...",1,"[-0.24427138, 0.36195734, 0.5969044, 0.1546584...",politicsNews,"December 30, 2017"
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,"[-0.6151011, 0.17262064, -0.023531837, 0.45550...",1,"[-0.24427138, 0.36195734, 0.5969044, 0.1546584...",politicsNews,"December 29, 2017"
...,...,...,...,...,...,...
20821,KABUL (Reuters) - A senior U.S. commander in A...,"[-0.08719649, 0.9996257, -0.5225119, 0.2537291...",0,"[-0.10262637, 0.5293654, 0.35757658, 0.0901561...",worldnews,"September 6, 2017"
20822,WASHINGTON (Reuters) - With no palatable milit...,"[-0.057866868, 0.6627015, -0.5182752, -0.19243...",0,"[-0.10262637, 0.5293654, 0.35757658, 0.0901561...",worldnews,"September 6, 2017"
20823,WASHINGTON (Reuters) - President Donald Trump ...,"[-0.0926037, 0.30926335, 1.3640616, 0.6173858,...",0,"[-0.10262637, 0.5293654, 0.35757658, 0.0901561...",worldnews,"September 6, 2017"
20824,KIGALI (Reuters) - Rwandan police have arreste...,"[-0.48847264, 0.21804309, 1.414586, 0.08558074...",1,"[-0.24427138, 0.36195734, 0.5969044, 0.1546584...",worldnews,"September 6, 2017"


In [16]:
sentence_df['text'] = true['text']
sentence_df['subject'] = true['subject']
sentence_df['date'] = true['date']
sentence_df['title'] = true['text']

sentence_df

,text,emb,cluster,centroid,subject,date
0,WASHINGTON (Reuters) - The head of a conservat...,"[-0.5992365, 0.45873383, -0.34349346, 0.103394...",0,"[-0.10262637, 0.5293654, 0.35757658, 0.0901561...",politicsNews,"December 31, 2017"
1,WASHINGTON (Reuters) - Transgender people will...,"[0.069638215, 0.53655505, 0.45924962, 0.224080...",1,"[-0.24427138, 0.36195734, 0.5969044, 0.1546584...",politicsNews,"December 29, 2017"
2,WASHINGTON (Reuters) - The special counsel inv...,"[-0.8980431, 0.84262395, -0.13861199, 0.363072...",1,"[-0.24427138, 0.36195734, 0.5969044, 0.1546584...",politicsNews,"December 31, 2017"
3,WASHINGTON (Reuters) - Trump campaign adviser ...,"[-0.0060276706, 0.3452367, -0.07653967, 0.5917...",1,"[-0.24427138, 0.36195734, 0.5969044, 0.1546584...",politicsNews,"December 30, 2017"
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,"[-0.6151011, 0.17262064, -0.023531837, 0.45550...",1,"[-0.24427138, 0.36195734, 0.5969044, 0.1546584...",politicsNews,"December 29, 2017"
...,...,...,...,...,...,...
20821,KABUL (Reuters) - A senior U.S. commander in A...,"[-0.08719649, 0.9996257, -0.5225119, 0.2537291...",0,"[-0.10262637, 0.5293654, 0.35757658, 0.0901561...",worldnews,"September 6, 2017"
20822,WASHINGTON (Reuters) - With no palatable milit...,"[-0.057866868, 0.6627015, -0.5182752, -0.19243...",0,"[-0.10262637, 0.5293654, 0.35757658, 0.0901561...",worldnews,"September 6, 2017"
20823,WASHINGTON (Reuters) - President Donald Trump ...,"[-0.0926037, 0.30926335, 1.3640616, 0.6173858,...",0,"[-0.10262637, 0.5293654, 0.35757658, 0.0901561...",worldnews,"September 6, 2017"
20824,KIGALI (Reuters) - Rwandan police have arreste...,"[-0.48847264, 0.21804309, 1.414586, 0.08558074...",1,"[-0.24427138, 0.36195734, 0.5969044, 0.1546584...",worldnews,"September 6, 2017"


In [14]:
true['emb'] = sentence_df['emb']
true['cluster'] = sentence_df['cluster']
true

,title,text,subject,date,emb,cluster
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017","[-0.5992365, 0.45873383, -0.34349346, 0.103394...",0.0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017","[0.069638215, 0.53655505, 0.45924962, 0.224080...",1.0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017","[-0.8980431, 0.84262395, -0.13861199, 0.363072...",1.0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017","[-0.0060276706, 0.3452367, -0.07653967, 0.5917...",1.0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017","[-0.6151011, 0.17262064, -0.023531837, 0.45550...",1.0
...,...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",NaN,NaN
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",NaN,NaN
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",NaN,NaN
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",NaN,NaN


In [20]:
array[:20]

0     As U.S. budget fight looms, Republicans flip t...
14    Man says he delivered manure to Mnuchin to pro...
15    Virginia officials postpone lottery drawing to...
18    U.S. appeals court rejects challenge to Trump ...
21    Exclusive: U.S. memo weakens guidelines for pr...
22    Trump travel ban should not apply to people wi...
23    Second court rejects Trump bid to stop transge...
24    Failed vote to oust president shakes up Peru's...
35    Exclusive: State Department tells refugee agen...
42    U.S. court rejects Trump bid to stop transgend...
45    Second U.S. judge blocks Trump administration ...
46    Senators seek to stop expansion of airport fac...
47    U.S. launches effort to reduce reliance on imp...
54    Trump commutes fraud sentence of kosher meatpa...
55    U.S. responds in court fight over illegal Indo...
56        Democrat Franken to leave Senate on January 2
58    Top Democrat says Trump firing of Mueller coul...
62    Democrats plan to use tax bill to attack R

In [8]:
from nltk.cluster import KMeansClusterer
import nltk

def clustering_question(data,NUM_CLUSTERS = 20):

    sentences = data['text']

    X = np.array(data['emb'].tolist())

    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=25,avoid_empty_clusters=True)

    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

    data['cluster'] = pd.Series(assigned_clusters, index=data.index)
    data['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])

    return data, assigned_clusters

def distance_from_centroid(row):
    # type of emb and centroid is different, hence using tolist below
    return distance_matrix([row['emb']], [row['centroid'].tolist()])[0][0]

# Compute centroid distance to the data
#sentence_df['distance_from_centroid'] = sentence_df.apply(distance_from_centroid, axis=1)

clustering_question(sentence_df)

(                                                    text  \
 0      As U.S. budget fight looms, Republicans flip t...   
 1      U.S. military to accept transgender recruits o...   
 2      Senior U.S. Republican senator: 'Let Mr. Muell...   
 3      FBI Russia probe helped by Australian diplomat...   
 4      Trump wants Postal Service to charge 'much mor...   
 ...                                                  ...   
 20821  North Korea shipments to Syria chemical arms a...   
 20822  LexisNexis withdrew two products from Chinese ...   
 20823  Minsk cultural hub becomes haven from authorities   
 20824  Vatican upbeat on possibility of Pope Francis ...   
 20825  Indonesia to buy $1.14 billion worth of Russia...   
 
                                                      emb  cluster  \
 0      [-0.5992365, 0.45873383, -0.34349346, 0.103394...        8   
 1      [0.069638215, 0.53655505, 0.45924962, 0.224080...        4   
 2      [-0.8980431, 0.84262395, -0.13861199, 0.363072..

In [ ]:
print(data['cluster'].eq.0)

In [14]:
filter = sentence_df.cluster.eq(1)
#print(sentence_df[filter]['text'])

array = sentence_df[filter]['text']
print(array.values)
print(len(array))

['U.S. appeals court rejects challenge to Trump voter fraud panel'
 'Trump travel ban should not apply to people with strong U.S. ties: court'
 'Second court rejects Trump bid to stop transgender military recruits'
 ... "Excessive force won't solve Myanmar's Rohingya crisis: Annan panel"
 "Pakistan rejects role of 'scapegoat for U.S. failures' in Afghanistan"
 'Romanian president opposes plans for judicial overhaul']
1308


In [ ]:
csvpath = 'C:/Users/katya/Fake_News/News_dataset/sentence_df'
sentence_df.to_csv(csvpath)